<a href="https://colab.research.google.com/github/tldtldcpfl/JoChaeRin/blob/master/KG_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

지식그래프 임베딩

- WordNet을 이용하여 단어를 생성하고 임베딩을 진행한다.




scikit-kge를 설치한다. 

In [ ]:
# make sure we are in the right folder to perform the git clone
%cd /content/
!git clone https://github.com/hybridNLP2018/scikit-kge

/content
Cloning into 'scikit-kge'...
remote: Enumerating objects: 116, done.
remote: Total 116 (delta 0), reused 0 (delta 0), pack-reused 116
Receiving objects: 100% (116/116), 25.32 KiB | 2.53 MiB/s, done.
Resolving deltas: 100% (51/51), done.


In [ ]:
%cd scikit-kge
# install a dependency of scikit-kge on the colaboratory environment, needed to correclty build scikit-kge
!pip install nose
# now build a source distribution for the project
!python setup.py sdist

/content/scikit-kge
     |████████████████████████████████| 163kB 2.8MB/s 
setuptools module not found.
Install setuptools if you want to enable 'python setup.py develop'.
running sdist
running egg_info
/usr/lib/python3.6/distutils/dist.py:974: UserWarning: 
`build_src` is being run, this may lead to missing
files in your sdist!  You want to use distutils.sdist
instead of the setuptools version:

    from distutils.command.sdist import sdist
    cmdclass={'sdist': sdist}"

See numpy's setup.py or gh-7131 for details.
  cmd_obj.run()
running build_src
build_src
creating scikit_kge.egg-info
writing scikit_kge.egg-info/PKG-INFO
writing dependency_links to scikit_kge.egg-info/dependency_links.txt
writing top-level names to scikit_kge.egg-info/top_level.txt
writing manifest file 'scikit_kge.egg-info/SOURCES.txt'
writing manifest file 'scikit_kge.egg-info/SOURCES.txt'
running check
creating scikit-kge-0.1
creating scikit-kge-0.1/skge
creating scikit-kge-0.1/scikit_kge.egg-info
copying files 

In [ ]:
!ls dist/

scikit-kge-0.1.tar.gz


In [ ]:
!pip install dist/scikit-kge-0.1.tar.gz
%cd /content

Processing ./dist/scikit-kge-0.1.tar.gz
  Created wheel for scikit-kge: filename=scikit_kge-0.1-cp36-none-any.whl size=12515 sha256=c21269f9d7057849bd34f4475fe020983917525b297a27e21827e33549c39be7
  Stored in directory: /root/.cache/pip/wheels/fb/75/c7/6e9f169770d70852cb44e65ddc7a7ac93c2dfba1ebee685b17
Successfully built scikit-kge
/content


In [ ]:
# let's go back to the main \content folder and clone the holE repo
%cd /content/
!git clone https://github.com/mnick/holographic-embeddings

/content
Cloning into 'holographic-embeddings'...
remote: Enumerating objects: 37, done.
remote: Total 37 (delta 0), reused 0 (delta 0), pack-reused 37
Unpacking objects: 100% (37/37), done.


In [ ]:
%less holographic-embeddings/run_hole_wn18.sh

graph를 serialized python 딕셔너리 형태로 바꾼다. 

In [ ]:
import pickle
import os

with open('holographic-embeddings/data/wn18.bin', 'rb') as fin:
  wn18_data = pickle.load(fin)

for k in wn18_data:
  print(k, type(wn18_data[k]), len(wn18_data[k]), wn18_data[k][-3:])

relations <class 'list'> 18 ['_synset_domain_region_of', '_verb_group', '_similar_to']
train_subs <class 'list'> 141442 [(5395, 37068, 9), (5439, 35322, 11), (28914, 1188, 10)]
entities <class 'list'> 40943 ['01164618', '02371344', '03788703']
test_subs <class 'list'> 5000 [(17206, 33576, 0), (1179, 11861, 0), (30287, 1443, 1)]
valid_subs <class 'list'> 5000 [(351, 25434, 0), (3951, 2114, 7), (756, 14490, 0)]


In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from nltk.corpus import wordnet as wn

In [ ]:
wn.synsets('dog')

[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01'),
 Synset('chase.v.01')]

In [ ]:
for synset in list(wn.all_synsets())[:5]:
    print(synset.name())

able.a.01
unable.a.01
abaxial.a.01
adaxial.a.01
acroscopic.a.01


In [ ]:
for lemma in list(wn.all_lemma_names())[5000:5005]:
    print(lemma)

cystic
cytoarchitectonic
cytoarchitectural
cytogenetic
cytogenetical


In [ ]:
wn.synset('adaxial.a.01').topic_domains()

[Synset('biology.n.01')]

In [ ]:
wn.synset('adaxial.a.01').lemmas()

[Lemma('adaxial.a.01.adaxial'), Lemma('adaxial.a.01.ventral')]

In [ ]:
wn.synset('adaxial.a.01').lemma_names()

['adaxial', 'ventral']

In [ ]:
syn_relations = {
    'hyponym': lambda syn: syn.hyponyms(), 
    'instance_hyponym': lambda syn: syn.instance_hyponyms(),  
    'member_meronym': lambda syn: syn.member_meronyms(),
    'has_part': lambda syn: syn.part_meronyms(), 
    'topic_domain': lambda syn: syn.topic_domains(), 
    'usage_domain': lambda syn: syn.usage_domains(), 
    '_member_of_domain_region': lambda syn: syn.region_domains(),
    'attribute': lambda syn: syn.attributes(),
    'entailment': lambda syn: syn.entailments(),
    'cause': lambda syn: syn.causes(),
    'also_see': lambda syn: syn.also_sees(),
    'verb_group': lambda syn: syn.verb_groups(),
    'similar_to': lambda syn: syn.similar_tos()
}
lem_relations = {
    'antonym': lambda lem: lem.antonyms(),
    'derivationally_related_form': lambda lem: lem.derivationally_related_forms(),
    'pertainym': lambda lem: lem.pertainyms()
}

syn2lem_relations = {
    'lemma': lambda syn: syn.lemma_names()
}

In [ ]:
def generate_syn_triples(entity_id_map, rel_id_map):
  result = []
  for synset in list(wn.all_synsets()):
    h_id = entity_id_map.get(synset.name())
    if h_id is None:
      print('No entity id for ', synset)
      continue
    for synrel, srfn in syn_relations.items():
      r_id = rel_id_map.get(synrel)
      if r_id is None:
        print('No rel id for', synrel)
        continue
      for obj in srfn(synset):  
        t_id = entity_id_map.get(obj.name())
        if t_id is None:
          print('No entity id for object', obj)
          continue
        result.append((h_id, t_id, r_id))
    
    for rel, fn in syn2lem_relations.items():
      r_id = rel_id_map.get(rel)
      if r_id is None:
        print('No rel id for', rel)
        continue
      for obj in fn(synset):
        lem = obj.lower()
        t_id = entity_id_map.get(lem)
        if t_id is None:
          print('No entity id for object', obj, 'lowercased:', lem)
          continue
        result.append((h_id, t_id, r_id))
  return result

In [ ]:
def generate_lem_triples(entity_id_map, rel_id_map):
  result = []
  for lemma in list(wn.all_lemma_names()):
    h_id = entity_id_map.get(lemma)
    if h_id is None:
      print('No entity id for lemma', lemma)
      continue
    _lems = wn.lemmas(lemma)
    for lemrel, lrfn in lem_relations.items():
      r_id = rel_id_map.get(lemrel)
      if r_id is None:
        print('No rel id for ', lemrel)
        continue
      for _lem in _lems:
        for obj in lrfn(_lem):
          t_id = entity_id_map.get(obj.name().lower())
          if t_id is None:
            print('No entity id for obj lemma', obj, obj.name())
            continue
          result.append((h_id, t_id, r_id))
  return result

In [ ]:
import random # for shuffling list of triples
      
def wnet30_holE_bin(out):
  """Creates a skge-compatible bin file for training HolE embeddings based on WordNet31"""
  synsets = [synset.name() for synset in wn.all_synsets()]
  lemmas = [lemma for lemma in wn.all_lemma_names()]
  entities = list(synsets + list(set(lemmas)))
  print('Found %s synsets, %s lemmas, hence %s entities' % (len(synsets), len(lemmas), len(entities)))
  entity_id_map = {ent_name: id for id, ent_name in enumerate(entities)}
  n_entity = len(entity_id_map)
    
  print("N_ENTITY: %d" % n_entity)
    
  relations = list( list(syn_relations.keys()) + list(lem_relations.keys()) + list(syn2lem_relations.keys()))
  relation_id_map = {rel_name: id for id, rel_name in enumerate(relations)}
  n_rel = len(relation_id_map)
    
  print("N_REL: %d" % n_rel)
  print('relations', relation_id_map)
    
  syn_triples = generate_syn_triples(entity_id_map, relation_id_map)
  print("Syn2syn relations", len(syn_triples))
  lem_triples = generate_lem_triples(entity_id_map, relation_id_map)
  print("Lem2lem relations", len(lem_triples))
  all_triples = syn_triples + lem_triples
  print("All triples", len(all_triples))
  random.shuffle(all_triples)
    
  test_triple = all_triples[:500]
  valid_triple = all_triples[500:1000]
  train_triple = all_triples[1000:]
        
  to_pickle = {
      "entities": entities,
      "relations": relations,
      "train_subs": train_triple,
      "test_subs": test_triple,
      "valid_subs": valid_triple
  }
    
  with open(out, 'wb') as handle:
    pickle.dump(to_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
  print("wrote to %s" % out)

In [ ]:
with open('/content/holographic-embeddings/data/wn30.bin', 'rb') as fin:
  wn30_data = pickle.load(fin)
entities = wn30_data['entities']
len(entities)

264965